In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 28973557
paper_name = 'delarosa_vulpe_2017' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [18]:
original_data = pd.read_excel('raw_data/TableS1.xlsx', sheet_name='Sheet1', skiprows=2)

In [19]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 211 x 6


In [20]:
original_data.head()

,Unnamed: 0,Unnamed: 1,4.5 µM,9 µM,18 µM,Unnamed: 5
0,YIL162W,SUC2,-4.2,-4.1,-4.00,3
1,YFR036W,CDC26,-3.2,-3.8,-3.05,3
2,YDR112W,IRC2,-2.7,-4.0,-2.95,3
3,YLR032W,RAD5,NaN,-2.2,-2.70,2
4,YCR066W,RAD18,NaN,-2.1,-2.40,2


In [21]:
original_data['orf'] = original_data['Unnamed: 0'].astype(str)

In [22]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [23]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [24]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 1, 4.5 µM, 9 µM, 18 µM, Unnamed: 5, orf]
Index: []


In [25]:
original_data.set_index('orf', inplace=True)

In [26]:
original_data = original_data[['4.5 µM','9 µM','18 µM']].copy()

In [27]:
original_data[original_data.isnull()] = 0

In [28]:
original_data = original_data.groupby(original_data.index).mean()

In [29]:
original_data.shape

(211, 3)

# Prepare the final dataset

In [30]:
data = original_data.copy()

In [31]:
dataset_ids = [16696,16697,16698]
datasets = datasets.reindex(index=dataset_ids)

In [32]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [33]:
data.head()

dataset_id,16696,16697,16698
data_type,value,value,value
orf,,,
YAL002W,0.0,-1.8,-2.3
YAL024C,0.0,0.0,-1.7
YBL008W,0.0,1.7,0.0
YBL062W,0.0,2.4,0.0
YBL107C,0.0,1.7,1.6


## Subset to the genes currently in SGD

In [34]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [35]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16696,16697,16698
,data_type,value,value,value
gene_id,orf,,,
2,YAL002W,0.0,-1.8,-2.3
22,YAL024C,0.0,0.0,-1.7
96,YBL008W,0.0,1.7,0.0
150,YBL062W,0.0,2.4,0.0
195,YBL107C,0.0,1.7,1.6


# Normalize

In [36]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [37]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [38]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16696,16697,16698,16696,16697,16698
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
2,YAL002W,0.0,-1.8,-2.3,-0.070802,-4.371014,-5.926145
22,YAL024C,0.0,0.0,-1.7,-0.070802,0.022188,-4.396319
96,YBL008W,0.0,1.7,0.0,-0.070802,4.171323,-0.061811
150,YBL062W,0.0,2.4,0.0,-0.070802,5.879791,-0.061811
195,YBL107C,0.0,1.7,1.6,-0.070802,4.171323,4.017725


# Print out

In [39]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [40]:
from IO.save_data_to_db3 import *

In [41]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 28973557...
Inserting the new data...


100%|██████████| 3/3 [00:01<00:00,  2.36it/s]

Updating the data_modified_on field...
